In [ ]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import seaborn as sns
import pandas as pd
sns.set_context('talk')

sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
cal_exp = '03_07_24_RandomCircles_Calibrators'

path_list = [
    f"{root}/calibration/{cal_exp}/Individual_TempScaling",
    f"{root}/calibration/{cal_exp}/Individual_LTS",
]

df_list = []
for path in path_list:
    dfc = rs.load_configs(
        path,
        properties=False,
    )
    df = rs.load_metrics(dfc)
    df_list.append(df)

In [ ]:
df = pd.concat(df_list)

In [ ]:
def calibrator(model):
    return model.split('.')[-1]

df.augment(calibrator)

In [ ]:
df.keys()

In [ ]:
df.head()

In [ ]:
# def cal_cfg(calibrator, use_image, use_logits, filters):
#     if calibrator == "Temperature_Scaling":
#         return calibrator
#     else:
#         return f"{calibrator}_useimage:{use_image}_uselogits:{use_logits}_filters:{filters}"

# df.augment(cal_cfg)

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss',
    col='phase',
    hue='calibrator',
    kind='line',
    height=12,
    )

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
g.set(ylim=(0.0, 0.25))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='ece_loss',
    col='phase',
    hue='calibrator',
    kind='line',
    height=12,
    )

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# g.set(ylim=(0.0, 0.25))